In [460]:
import pandas as pd
import altair as alt
alt.renderers.set_embed_options(theme='dark')


RendererRegistry.enable('default')

In [461]:
def claim(claim_hash, cols_claim):
    try:
        df_claim = pd.read_json(
            f"https://api.flipsidecrypto.com/api/v2/queries/{claim_hash}/data/latest",
            convert_dates=["BLOCK_TIMESTAMP"],
        )
    except:
        return pd.DataFrame(columns = cols_claim[claim_hash])
    if(len(df_claim.columns)==0):
        return pd.DataFrame(columns = cols_claim[claim_hash])
    return df_claim

In [462]:
p2_users = '07f87a6f-5bc6-49f3-a1bb-8e7d545ba95a'
p2_hourly = 'de5232e1-5d57-4232-900d-1d09d2c9438e'

In [463]:
cols_claim = {
    p2_users: ["DEPOSIT_AMOUNT","HR","NET_DEPOSITED_AMOUNT","SENDER","WITHDRAWN_AMOUNT","WITHDRAWN_AMOUNT_PHASE2"],
    p2_hourly: ['HR', 'WITH_AMOUNT', 'WITH_TX', 'WITH_USERS']
}

In [464]:
user_stats = '499224b4-30a6-43d7-80b9-3a019cbb1d3d'
user_stats_df = claim(user_stats, cols_claim)

In [465]:
user_stats_df.columns = [c.lower() for c in user_stats_df.columns]
user_stats_df

,deposit_amount,deposit_txs,sender,withdrawn_amount,withdraw_txs
0,200.000,2,terra1u793r2hfd684ae3ymyrts3jgtfl5ur40j9f07t,0.0,0
1,150.000,1,terra15sxyjyzyz3k2gsyh8skp86lvtl7xj8fys5hrth,0.0,0
2,225.000,1,terra1n8rrqld0tgguqle30n2cg2xfxguuevhkf8kkth,0.0,0
3,1800.000,1,terra1t3gtrly507dw8fsxzqzyjs2zefm07a7ugkw0ka,0.0,0
4,96.896,1,terra1xhdpqx8laux3lgfm4amfqdtgyevuaxerezknye,0.0,0
...,...,...,...,...,...
5468,1000.000,1,terra17npp9fq9uzanv05rytgj86muaet59rpth55aln,0.0,0
5469,2500.000,1,terra10lwmun98vpv9fuup2jnjvwp70a9778ra8tqjpm,0.0,0
5470,100.000,1,terra1upp95s4gh0l7cpm7e5esw4kdpwwm68c8xnvtwh,0.0,0
5471,50.000,1,terra1f78fdgp380zq0n4fajv3vla0azu79z09kv7gza,0.0,0


In [466]:
def get_url(url):
    return pd.read_csv(url, index_col=0)

In [467]:
with_phase1 = get_url('https://raw.githubusercontent.com/IncioMan/prism_forge/p22/data/with_phase1.csv')
with_phase1

,sender,deposit,total_deposit,withdrawable_amount,tokens_to_claim,can_claim
0,terra15r46lszvnt8eav4q22m0x2wljkvpxw7c3x554u,200.000000,2.522181e+07,200.000000,555.075144,False
1,terra1vdr5cq2zgk8960s3pf3tex490jq48lvkjz72lu,15026.495364,2.522181e+07,15026.495364,41704.170395,False
2,terra1j4ce0n3rs7my20080990kl8tmw9kngkzu8tn6w,1000.000000,2.522181e+07,1000.000000,2775.375720,False
3,terra1kvr2cznt8ycztcg5xjxex8uyqlz9lzm8ysf2f4,3000.000000,2.522181e+07,3000.000000,8326.127161,False
4,terra19vnxe0k6wwdv9j8al4uj2st9p9u02jdfvtvtm5,1000.000000,2.522181e+07,1000.000000,2775.375720,False
...,...,...,...,...,...,...
5226,terra1mwhgxr8u8s57v8qnqa053wunmhr2l6p0w6vc5a,301.000000,2.511240e+07,288.458333,839.027624,False
5227,terra1vlfcftd9m9uk2g0dd8epfnsvddamp3qxfzcy26,300.000000,2.511240e+07,287.499999,836.240157,False
5228,terra1xjuec7dfm8mlj28y9x9tau3ufr7vw3anagcefs,11500.000000,2.511240e+07,11020.833333,32055.872692,False
5229,terra1yhxpu2e2cv8ftw8ms95l6rmwklkfcuqd6m2gcy,80.000000,2.511240e+07,76.666666,222.997375,False


In [468]:
p2_users_df = claim(p2_users, cols_claim)
p2_hourly_df = claim(p2_hourly, cols_claim)

In [469]:
p2_hourly_df

,HR,WITH_AMOUNT,WITH_TX,WITH_USERS
0,02/05 08:00,119998.948418,15,15
1,02/05 07:00,57928.803221,35,35


In [470]:
p2_users_df.columns = [c.lower() for c in p2_users_df.columns]
p2_users_df

,deposit_amount,hr,net_deposited_amount,sender,withdrawn_amount,withdrawn_amount_phase2
0,20.0,NaN,20.0,terra1j3n7hyn2jang2a30lds92u5cwm54ywd509k4n5,0.0,NaN
1,11952.0,NaN,11952.0,terra1yxf7hesn7rcsle2hv3rzuavc3yr66h8l5sgkvr,0.0,NaN
2,300.0,NaN,300.0,terra1ywlexfcmerdg4dh5z0lg7e0yje8x9av752ul2h,0.0,NaN
3,1000.0,NaN,1000.0,terra1g9yr9grfxuwas9tmsc64g9ggktam4ndtpqrg69,0.0,NaN
4,4500.0,NaN,4500.0,terra1t4d6gwxc05z3xpy8j56mhcdz9k8sys0qt2xxdn,0.0,NaN
...,...,...,...,...,...,...
5226,680.0,NaN,680.0,terra1y4p7x7u0gltecy4damlkpjplapd7f7hnn4guuj,0.0,NaN
5227,450.0,NaN,450.0,terra18sy57rzasc2hmhxj38f5rv2eeagmjap59tus80,0.0,NaN
5228,1000.0,NaN,1000.0,terra1wjnn5sc7celp8pcf3kvuc88k825xfkuc0wu5kg,0.0,NaN
5229,50.0,NaN,50.0,terra1flmc4p8kcgswnwxyhnjx73etn8xkfsgfsjup56,0.0,NaN


In [471]:
users_with = p2_users_df.merge(with_phase1,on='sender')[['sender','net_deposited_amount','deposit','withdrawable_amount']]
users_with=users_with.rename(columns={'net_deposited_amount':'deposited_p1'})
users_with['WITHDRAWN_AMOUNT_PHASE2'] = users_with.deposited_p1-users_with.deposit
users_with['has_withdrawn_p2'] = (users_with['WITHDRAWN_AMOUNT_PHASE2'])>1
users_with

,sender,deposited_p1,deposit,withdrawable_amount,WITHDRAWN_AMOUNT_PHASE2,has_withdrawn_p2
0,terra1j3n7hyn2jang2a30lds92u5cwm54ywd509k4n5,20.0,20.0,20.000000,0.0,False
1,terra1yxf7hesn7rcsle2hv3rzuavc3yr66h8l5sgkvr,11952.0,11952.0,11952.000000,0.0,False
2,terra1ywlexfcmerdg4dh5z0lg7e0yje8x9av752ul2h,300.0,300.0,300.000000,0.0,False
3,terra1g9yr9grfxuwas9tmsc64g9ggktam4ndtpqrg69,1000.0,1000.0,1000.000000,0.0,False
4,terra1t4d6gwxc05z3xpy8j56mhcdz9k8sys0qt2xxdn,4500.0,4500.0,4500.000000,0.0,False
...,...,...,...,...,...,...
5226,terra1y4p7x7u0gltecy4damlkpjplapd7f7hnn4guuj,680.0,680.0,651.666666,0.0,False
5227,terra18sy57rzasc2hmhxj38f5rv2eeagmjap59tus80,450.0,450.0,450.000000,0.0,False
5228,terra1wjnn5sc7celp8pcf3kvuc88k825xfkuc0wu5kg,1000.0,1000.0,958.333333,0.0,False
5229,terra1flmc4p8kcgswnwxyhnjx73etn8xkfsgfsjup56,50.0,50.0,47.916666,0.0,False


In [472]:
users_with[users_with.has_withdrawn_p2]

,sender,deposited_p1,deposit,withdrawable_amount,WITHDRAWN_AMOUNT_PHASE2,has_withdrawn_p2
194,terra1g6qc9pwh7sy6jtrkd7fnzcs4j5z5spemey95ya,4996.000000,2996.000000,0.000000,2000.00000,True
233,terra1ytk0hphc86fl256g5l5s9ea4cqspdgskctdtgh,200.000000,0.000000,0.000000,200.00000,True
371,terra12nskn56mcenhe6lvuuzxzhdjstk8yy6pa3spu4,250.000000,0.000000,0.000000,250.00000,True
475,terra1uscncgg7kcea2f07l2r2l8g75kda2hcpvz9d3v,500.000000,0.000000,0.000000,500.00000,True
577,terra1phvjcgqf7530xzjjs9ahznqj72r3jwlsvz3ndp,500.000000,250.000000,250.000000,250.00000,True
587,terra1p9p2rnfhrt45gtfmrxj3t06kr5zdzak26na334,496.000000,0.000000,0.000000,496.00000,True
674,terra1wu3ksrardhm2zf6t8zcctwgmpyzwl55aqdfny0,300.000000,100.000000,100.000000,200.00000,True
810,terra1cr8gvqj86duhgfpvjs5x237v4wfsf0qykacsjx,20000.000000,10000.000000,9583.333333,10000.00000,True
1100,terra1jk4r2xxj6myl9vmsu9wvl498m9hf3mw9t9770c,1257.737000,0.000000,0.000000,1257.73700,True
1157,terra14nxj7tyqrndae3c2u8k8r2pcfzyh9pg6q07jz9,1000.000000,0.000000,0.000000,1000.00000,True


In [473]:
cols_dict = {
    "DEPOSIT_AMOUNT":"Amount deposited",
    "DEPOSIT_TXS":"N° of deposit txs",
    "DEP_USERS":"N° of depositing users",
    "HR":"Time",
    "NET_AMOUNT":"Net amount deposited",
    "TOT_TXS":"N° of txs",
    "TOT_USERS":"N° of users",
    "WITH_AMOUNT":"Withdrawn amount",
    "WITH_TXS":"N° of withdraw txs",
    "WITH_USERS":"N° of withdrawing users",
    'AMOUNT':'Amount deposited (UST)',
    'AVG_BALANCE_USD':'Average balance in the last 30 days (UST)',
    'N_TXS':'N° of txs',
    'SENDER':'User address',
    'N_USERS':'N° of users',
    'BUCKET':'Deposit range',
    'bucket_name':'Deposit bucket',
    'cumsum_ust':'Cumulative UST amount',
    'MIN_DATE':'Date wallet creation',
    'ADDRESS_COUNT':'Number of wallets',
    'PERC_WITHDRAWN':'% withdrawn of the deposited amount',
    'net_ust': 'Amount UST',
    'perc_withdrawn_cat':'Percentage withdrawn UST bucket',
    'DEP_CAT':'Deposited UST range'
}

In [474]:
#Total withdrawn phase 2 UST
users_with['WITHDRAWN_AMOUNT_PHASE2'] = users_with['WITHDRAWN_AMOUNT_PHASE2'].apply(lambda x: x if x >0 else 0)
tot_with_ust = users_with['WITHDRAWN_AMOUNT_PHASE2'].sum()
tot_with_ust

279923.62966000004

In [475]:
#Total net dep phase 1 UST
tot_ust_p1 = users_with['deposited_p1'].sum()
tot_ust_p1

21575401.345275998

In [476]:
#Total left UST
tot_net_ust = users_with.deposit.sum()
tot_net_ust

23997077.772152998

In [477]:
#% withdrawn
perc_with_p2 = tot_with_ust/tot_ust_p1 * 100
perc_with_p2

1.2974202666282737

In [478]:
#current price
curr_price = tot_net_ust/70000000
curr_price

0.3428153967450428

In [479]:
#% users with
p_users_with_p2 = (users_with.has_withdrawn_p2.sum()/len(p2_users_df))*100
p_users_with_p2

0.9176065761804627

In [480]:
p2_users_df.head()

,deposit_amount,hr,net_deposited_amount,sender,withdrawn_amount,withdrawn_amount_phase2
0,20.0,NaN,20.0,terra1j3n7hyn2jang2a30lds92u5cwm54ywd509k4n5,0.0,NaN
1,11952.0,NaN,11952.0,terra1yxf7hesn7rcsle2hv3rzuavc3yr66h8l5sgkvr,0.0,NaN
2,300.0,NaN,300.0,terra1ywlexfcmerdg4dh5z0lg7e0yje8x9av752ul2h,0.0,NaN
3,1000.0,NaN,1000.0,terra1g9yr9grfxuwas9tmsc64g9ggktam4ndtpqrg69,0.0,NaN
4,4500.0,NaN,4500.0,terra1t4d6gwxc05z3xpy8j56mhcdz9k8sys0qt2xxdn,0.0,NaN


In [481]:
df = pd.DataFrame([[tot_ust_p1-tot_with_ust,tot_with_ust],['Still deposited','Withdrawn']]).T
df.columns = ['UST','Type']
df

,UST,Type
0,2.12955e+07,Still deposited
1,279924,Withdrawn


In [482]:
alt.Chart(df).mark_arc().encode(
    theta=alt.Theta(field="UST", type="quantitative"),
    color=alt.Color(field="Type", type="nominal",
            scale=alt.Scale(scheme='pastel1'),
            legend=alt.Legend(
            orient='none',
            padding=10,
            legendY=-10,
            direction='vertical'))
).configure_view(strokeOpacity=0)

alt.Chart(...)

In [483]:
#UST left to be withdrawn
left_to_with = users_with.withdrawable_amount.sum()
left_to_with

23759614.311364003

In [484]:
tot_net_ust - left_to_with

237463.46078899503

In [485]:
#Floor price
floor_price = (tot_net_ust - left_to_with)/70000000
floor_price

0.0033923351541285006

In [486]:
p2_hourly_df['HR'] = '2022/' + p2_hourly_df['HR'] + ':00'
p2_hourly_df.head()

,HR,WITH_AMOUNT,WITH_TX,WITH_USERS
0,2022/02/05 08:00:00,119998.948418,15,15
1,2022/02/05 07:00:00,57928.803221,35,35


In [487]:
alt.Chart(p2_hourly_df.rename(columns=cols_dict)).mark_bar().encode(
    x=cols_dict['HR']+':T',
    y=cols_dict['WITH_AMOUNT']+":Q",
    tooltip=[cols_dict['HR']+':T',cols_dict['WITH_AMOUNT']+":Q"]
).configure_mark(
    color='#fab0ba'
).properties(width=700).configure_axisX(
    labelAngle=30
).configure_view(strokeOpacity=0)

alt.Chart(...)

In [488]:
alt.Chart(p2_hourly_df.rename(columns=cols_dict)).mark_bar().encode(
    x=cols_dict['HR']+':T',
    y=cols_dict['WITH_USERS']+":Q",
    tooltip=[cols_dict['HR']+':T',cols_dict['WITH_USERS']+":Q"]
).configure_mark(
    color='#fab0ba'
).properties(width=700).configure_axisX(
    labelAngle=30
).configure_view(strokeOpacity=0)

alt.Chart(...)

In [489]:
users_with.WITHDRAWN_AMOUNT_PHASE2 = users_with.WITHDRAWN_AMOUNT_PHASE2.apply(lambda x: x if x >=0 else 0)

In [536]:
with_users_df = users_with[users_with.has_withdrawn_p2&users_with['deposited_p1']>0]
with_users_df['perc_withdrawn'] = round(with_users_df[with_users_df.has_withdrawn_p2]['WITHDRAWN_AMOUNT_PHASE2'],3)/with_users_df['deposited_p1']*10
with_users_df['perc_withdrawn_cat'] = (with_users_df['perc_withdrawn']).apply(lambda x: int(x))
df2 = with_users_df.groupby('perc_withdrawn_cat').sender.count()
with_users_df

<ipython-input-536-1c290f1b25fc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  with_users_df['perc_withdrawn'] = round(with_users_df[with_users_df.has_withdrawn_p2]['WITHDRAWN_AMOUNT_PHASE2'],3)/with_users_df['deposited_p1']*10
<ipython-input-536-1c290f1b25fc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  with_users_df['perc_withdrawn_cat'] = (with_users_df['perc_withdrawn']).apply(lambda x: int(x))


,sender,deposited_p1,deposit,withdrawable_amount,WITHDRAWN_AMOUNT_PHASE2,has_withdrawn_p2,perc_withdrawn,perc_withdrawn_cat
194,terra1g6qc9pwh7sy6jtrkd7fnzcs4j5z5spemey95ya,4996.000000,2996.000000,0.000000,2000.00000,True,4.003203,4
233,terra1ytk0hphc86fl256g5l5s9ea4cqspdgskctdtgh,200.000000,0.000000,0.000000,200.00000,True,10.000000,10
371,terra12nskn56mcenhe6lvuuzxzhdjstk8yy6pa3spu4,250.000000,0.000000,0.000000,250.00000,True,10.000000,10
475,terra1uscncgg7kcea2f07l2r2l8g75kda2hcpvz9d3v,500.000000,0.000000,0.000000,500.00000,True,10.000000,10
577,terra1phvjcgqf7530xzjjs9ahznqj72r3jwlsvz3ndp,500.000000,250.000000,250.000000,250.00000,True,5.000000,5
587,terra1p9p2rnfhrt45gtfmrxj3t06kr5zdzak26na334,496.000000,0.000000,0.000000,496.00000,True,10.000000,10
674,terra1wu3ksrardhm2zf6t8zcctwgmpyzwl55aqdfny0,300.000000,100.000000,100.000000,200.00000,True,6.666667,6
810,terra1cr8gvqj86duhgfpvjs5x237v4wfsf0qykacsjx,20000.000000,10000.000000,9583.333333,10000.00000,True,5.000000,5
1100,terra1jk4r2xxj6myl9vmsu9wvl498m9hf3mw9t9770c,1257.737000,0.000000,0.000000,1257.73700,True,10.000000,10
1157,terra14nxj7tyqrndae3c2u8k8r2pcfzyh9pg6q07jz9,1000.000000,0.000000,0.000000,1000.00000,True,10.000000,10


In [537]:
perc_cat = list(range(1,11))
cat = pd.DataFrame([0]*10,perc_cat)
df3 = cat.join(df2,how='outer')
df3

,0,sender
1,0,1
2,0,1
3,0,1
4,0,1
5,0,7
6,0,6
7,0,2
8,0,4
9,0,1
10,0,24


In [538]:
perc_cat = list(range(1,11))
cat = pd.DataFrame([0]*10,perc_cat)
df3 = cat.join(df2,how='outer')
df3.index =  ['0%-10%','10%-20%','20%-30%','30%-40%','40%-50%','50%-60%','60%-70%','70%-80%','80%-90%','90%-100%']
df3 = df3.sender.fillna(0).reset_index()
with_perc_buckets=df3.rename(columns={'index':'PERC_WITHDRAWN','sender':'TOT_USERS'})

In [539]:
with_perc_buckets

,PERC_WITHDRAWN,TOT_USERS
0,0%-10%,1
1,10%-20%,1
2,20%-30%,1
3,30%-40%,1
4,40%-50%,7
5,50%-60%,6
6,60%-70%,2
7,70%-80%,4
8,80%-90%,1
9,90%-100%,24


In [540]:
alt.Chart(with_perc_buckets.sort_values(by='PERC_WITHDRAWN').rename(columns=cols_dict)).mark_bar().encode(
    y=alt.X(cols_dict['PERC_WITHDRAWN']+":N", sort=alt.EncodingSortField(order='ascending')),
    x=cols_dict['TOT_USERS']+":Q",
    tooltip=[cols_dict['PERC_WITHDRAWN']+":N",cols_dict['TOT_USERS']+":Q"]
).configure_mark(
    color='#F1705F'
).properties(height=300).configure_axisX(
    labelAngle=0
).configure_view(strokeOpacity=0)

alt.Chart(...)

In [541]:
p2_hourly_df = p2_hourly_df.sort_values(by='HR')
p2_hourly_df['cumsum_with'] = p2_hourly_df.sort_values(by='HR').WITH_AMOUNT.cumsum()
p2_hourly_df['net_ust'] = tot_ust_p1 - p2_hourly_df['cumsum_with']
p2_hourly_df.head()

,HR,WITH_AMOUNT,WITH_TX,WITH_USERS,cumsum_with,net_ust
1,2022/02/05 07:00:00,57928.803221,35,35,57928.803221,2.151747e+07
0,2022/02/05 08:00:00,119998.948418,15,15,177927.751639,2.139747e+07


In [542]:
alt.Chart(p2_hourly_df.rename(columns=cols_dict)).mark_line(point=True).encode(
    x=alt.X(cols_dict['HR']+':T', sort=alt.EncodingSortField(order='ascending')),
    y=cols_dict['net_ust']+":Q",
    tooltip=[alt.Tooltip(cols_dict['HR']+':T', format='%Y-%m-%d %H:%M'), alt.Tooltip(cols_dict['net_ust']+":Q")]
).configure_mark(
    color='#F1705F'
).properties(width=700).configure_axisX(
).configure_view(strokeOpacity=0)

alt.Chart(...)

In [543]:
with_users_df['perc_withdrawn_cat']

194      4
233     10
371     10
475     10
577      5
587     10
674      6
810      5
1100    10
1157    10
1179    10
1180    10
1414     3
1436     6
1438    10
1576     7
1737     5
1768     5
1783    10
1983     9
2122     6
2218     8
2266    10
2595     6
2611    10
2706     8
2834     6
3058     1
3209    10
3336    10
3350    10
3373    10
3455    10
3599    10
3937     5
3948     5
4076    10
4115    10
4186     5
4286    10
4523     8
4530    10
4570    10
4613    10
4832     6
4919     8
5037     2
5230     7
Name: perc_withdrawn_cat, dtype: int64

In [544]:
with_users_df['perc_withdrawn_cat'].apply(lambda x: int(int(x*10/5)))

194      8
233     20
371     20
475     20
577     10
587     20
674     12
810     10
1100    20
1157    20
1179    20
1180    20
1414     6
1436    12
1438    20
1576    14
1737    10
1768    10
1783    20
1983    18
2122    12
2218    16
2266    20
2595    12
2611    20
2706    16
2834    12
3058     2
3209    20
3336    20
3350    20
3373    20
3455    20
3599    20
3937    10
3948    10
4076    20
4115    20
4186    10
4286    20
4523    16
4530    20
4570    20
4613    20
4832    12
4919    16
5037     4
5230    14
Name: perc_withdrawn_cat, dtype: int64

In [545]:
with_users_df['DEP_CAT'] = (with_users_df['deposited_p1']/20000).apply(int)
with_users_df['perc_withdrawn_cat_old'] = with_users_df['perc_withdrawn_cat']
with_users_df['perc_withdrawn_cat'] = with_users_df['perc_withdrawn_cat'].apply(lambda x: int(int(x*10/5)))
df = with_users_df.groupby(['DEP_CAT','perc_withdrawn_cat']).sender.count()
df = df.reset_index()
df

<ipython-input-545-91276e1087f1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  with_users_df['DEP_CAT'] = (with_users_df['deposited_p1']/20000).apply(int)
<ipython-input-545-91276e1087f1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  with_users_df['perc_withdrawn_cat_old'] = with_users_df['perc_withdrawn_cat']
<ipython-input-545-91276e1087f1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

,DEP_CAT,perc_withdrawn_cat,sender
0,0,2,1
1,0,4,1
2,0,8,1
3,0,10,4
4,0,12,6
5,0,14,1
6,0,16,4
7,0,18,1
8,0,20,22
9,1,10,2


In [546]:
alt.Chart(df).mark_rect().encode(
    x=alt.X('perc_withdrawn_cat:O', sort=alt.EncodingSortField(order='ascending')),
    y=alt.Y('DEP_CAT:O', sort=alt.EncodingSortField(order='descending')),
    color='sender:Q'
)

alt.Chart(...)

In [547]:
df

,DEP_CAT,perc_withdrawn_cat,sender
0,0,2,1
1,0,4,1
2,0,8,1
3,0,10,4
4,0,12,6
5,0,14,1
6,0,16,4
7,0,18,1
8,0,20,22
9,1,10,2


In [548]:
heatmap_val

,DEP_CAT,DEP_CAT_label,perc_withdrawn_cat,perc_withdrawn_cat_label
0,0,0-20k,0,0%-5%
1,0,0-20k,5,5%-10%
2,0,0-20k,10,10%-15%
3,0,0-20k,15,15%-20%
4,0,0-20k,20,20%-25%
...,...,...,...,...
575,560,560-580k,75,75%-80%
576,560,560-580k,80,80%-85%
577,560,560-580k,85,85%-90%
578,560,560-580k,90,90%-95%


In [557]:
df1

,0,1,fake
0,1,0%-5%,0
1,2,5%-10%,0
2,3,10%-15%,0
3,4,15%-20%,0
4,5,20%-25%,0
5,6,25%-30%,0
6,7,30%-35%,0
7,8,35%-40%,0
8,9,40%-45%,0
9,10,45%-50%,0


In [558]:
import random
df1 = pd.DataFrame([list(range(1,21)),
                    ["0%-5%",	"5%-10%",	"10%-15%",	"15%-20%",	"20%-25%",	"25%-30%",	"30%-35%",	
"35%-40%",	"40%-45%",	"45%-50%",	"50%-55%",	"55%-60%",	"60%-65%",	
"65%-70%",	"70%-75%",	"75%-80%",	"80%-85%",	"85%-90%",	"90%-95%",	"95%-100%"]]).T
df2 = pd.DataFrame([list(range(0,561,20)),
                    ["0-20k", "20-40k", "40-60k", "60-80k", "80-100k", "100-120k", "120-140k", "140-160k", 
                     "160-180k", "180-200k","200-220k", "220-240k", "240-260k", "260-280k", "280-300k", "300-320k",
                     "320-340k",
                     "340-360k", 
                     "360-380k", "380-400k", "400-420k", "420-440k", "440-460k", "460-480k", "480-500k", "500-520k","520-540k",
                     "540-560k","560-580k"]]).T
df2['fake'] = 0
df1['fake'] = 0
heatmap_val = df2.merge(df1,on='fake').drop('fake',1)
heatmap_val.columns = ['DEP_CAT','DEP_CAT_label','perc_withdrawn_cat','perc_withdrawn_cat_label']
df = heatmap_val.merge(df, on=['perc_withdrawn_cat', 'DEP_CAT'], how='left').fillna(0)
heatmap_data_df = df.sort_values(by=['perc_withdrawn_cat', 'DEP_CAT'], ascending=[True,False])
heatmap_data_df


,DEP_CAT,DEP_CAT_label,perc_withdrawn_cat,perc_withdrawn_cat_label,DEP_CAT_label_x,perc_withdrawn_cat_label_x,DEP_CAT_label_y,perc_withdrawn_cat_label_y,sender
560,560,560-580k,1,0%-5%,560-580k,0%-5%,0,0,0.0
540,540,540-560k,1,0%-5%,540-560k,0%-5%,0,0,0.0
520,520,520-540k,1,0%-5%,520-540k,0%-5%,0,0,0.0
500,500,500-520k,1,0%-5%,500-520k,0%-5%,0,0,0.0
480,480,480-500k,1,0%-5%,480-500k,0%-5%,0,0,0.0
...,...,...,...,...,...,...,...,...,...
99,80,80-100k,20,95%-100%,80-100k,95%-100%,80-100k,20%-25%,0.0
79,60,60-80k,20,95%-100%,60-80k,95%-100%,60-80k,20%-25%,0.0
59,40,40-60k,20,95%-100%,40-60k,95%-100%,40-60k,20%-25%,0.0
39,20,20-40k,20,95%-100%,20-40k,95%-100%,20-40k,20%-25%,0.0


In [559]:
heatmap_data_df[heatmap_data_df.sender>0]

,DEP_CAT,DEP_CAT_label,perc_withdrawn_cat,perc_withdrawn_cat_label,DEP_CAT_label_x,perc_withdrawn_cat_label_x,DEP_CAT_label_y,perc_withdrawn_cat_label_y,sender
9,0,0-20k,10,45%-50%,0-20k,45%-50%,0-20k,10%-15%,4.0
19,0,0-20k,20,95%-100%,0-20k,95%-100%,0-20k,20%-25%,22.0


In [560]:
alt.Chart((heatmap_data_df.rename(columns=cols_dict))).mark_rect().encode(
    x=alt.X(cols_dict['perc_withdrawn_cat']+':O', sort=alt.EncodingSortField(order='ascending')),
    y=alt.Y(cols_dict['DEP_CAT']+':O', sort=alt.EncodingSortField(order='ascending')),
    color=alt.Color('sender'+':Q',
            scale=alt.Scale(scheme='redpurple')),
    tooltip=['DEP_CAT_label'+':N','perc_withdrawn_cat_label'+':N','sender'+':Q']
)

alt.Chart(...)